# core

> the core functionalities of fastdebug

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## make life easier with defaults  

In [ ]:
#| export
defaults = type('defaults', (object,), {'margin': 157, # align to the right by 157
                                        'orisrc': None, # keep a copy of original official src code
                                        'outenv': globals() # outside global env
                                        # 'eg': None, # examples
                                        # 'src': None, # official src
                                       }) 

## globals() and locals()

Interesting behavior of [locals()](https://stackoverflow.com/questions/7969949/whats-the-difference-between-globals-locals-and-vars)

In [ ]:
locals() == globals()

True

In [ ]:
"functools" in locals()

False

In [ ]:
"inspect" in locals()

False

In [ ]:
"eval" in locals()

False

In [ ]:
"defaults" in locals()

True

In [ ]:
"whatinside" in locals()

False

In [ ]:
"whichversion" in locals()

False

In [ ]:
from fastdebug.utils import *

In [ ]:
"whatinside" in locals()

True

In [ ]:
"whichversion" in globals()

True

## Execute strings

In [ ]:
eval?

Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
#| export
from pprint import pprint

### new variable or updated variable by exec will only be accessible from locals()

In [ ]:
x = 1
def test():
    a = "1+x"
    pprint(f'locals: {locals()}', width=157) # x is not in locals() but in globals()
    print(eval(a)) 
test()

"locals: {'a': '1+x'}"
2


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    print(b) # b can't be accessed directly and not use the key b

try:
    test()
except NameError as e:
    print(e)
    

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
name 'b' is not defined


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    b = locals()['b'] # you can't even assign the value to b, otherwise b:value won't be available
    print(b)

# test()
try:
    test()
except KeyError as e:
    print("KeyError: 'b' does not exist")

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x'}
KeyError: 'b' does not exist


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(c)
    pprint(locals())

# test()
try:
    test()
except KeyError as e:
    print("KeyError: 'b' does not exist")

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
2
{'a': 'b = 1+x', 'b': 2, 'c': 2}


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(f'c = locals()["b"]; c: {c}')
    pprint(locals())
    print(f'exec(c = c + 1): {exec("c = c + 1")}') # update c won't change anything
    pprint(locals())

test()

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
c = locals()["b"]; c: 2
{'a': 'b = 1+x', 'b': 2, 'c': 2}
exec(c = c + 1): None
{'a': 'b = 1+x', 'b': 2, 'c': 2}


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(f'c = locals()["b"]; c: {c}')
    pprint(locals())
    print(f'exec(d = c + 1): {exec("d = c + 1")}') # must assign to a different variable, not c anymore
    pprint(locals())

test()

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
c = locals()["b"]; c: 2
{'a': 'b = 1+x', 'b': 2, 'c': 2}
exec(d = c + 1): None
{'a': 'b = 1+x', 'b': 2, 'c': 2, 'd': 3}


### eval can override its own globals() and locals()

In [ ]:
def test():
    a = 1
    b = "a+1"
    pprint(f'locals: {locals()}', width=157)
    print(f'b: {eval(b, {}, {"a": 2})}') # globals() put to empty and locals() to include 'a' with a different value
    pprint(f'locals: {locals()}', width=157) 

test()

"locals: {'a': 1, 'b': 'a+1'}"
b: 3
"locals: {'a': 1, 'b': 'a+1'}"


### when exec update existing functions

In [ ]:
#| export
import inspect

In [ ]:
def foo(x, y): return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"]
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)    

"locals: {'func': <function foo>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': ''}"
exec(newb): None
("locals: {'func': <function foo>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': '', "
 "'foo': <function foo>}")
13
13
10
10


### when the func to be udpated involve other libraries

In [ ]:
import functools

In [ ]:
def foo(x, y): 
    print(inspect.signature(foo))
    return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l) and "return" not in l :
            newb = newb + l + '\n'
        elif bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"]
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)    

("locals: {'func': <function foo>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': ''}")
exec(newb): None
("locals: {'func': <function foo>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': '', 'foo': <function foo>}")
(x, y)
13
(x, y)
13
(x, y)
10
(x, y)
10


### inside a function, exec() allow won't give you necessary env from function namespace

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    exec(b) # create the new add in locals
    pprint(f'locals: {locals()}', width=157)
    print(f'add(5,6): {add(5,6)}')
    add1 = locals()['add'] # assign a different name, add1
    
    print(f'add1(5,6): {add1(5,6)}') # error: lst is not defined, even though lst=[] is right above
    
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

"locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x118b54700>}')
add(5,6): 1
name 'lst' is not defined


### magic of `exec(b, globals().update(locals()))`

What about `exec(b, globals().update(globals()))`

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(globals())) # update(globals()) won't give us lst above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)



try:
    test(add)
except: 
    print("exec(b, globals().update(globals())) won't give us lst in the func namespace")

"locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x118b549d0>}')
exec(b, globals().update(globals())) won't give us lst in the func namespace


In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(locals())) # make sure b can access lst from above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)

test(add)
print(add(5,6))
try:
    print(add1(5,6))
except: 
    print("you can't bring add1 from a function namespace to the outside world")

"locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x118b54dc0>}')
11
("locals: {'func': <function add>, 'a': 1, 'lst': [5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 "0x118b54dc0>, 'add1': <function add>}")
1
you can't bring add1 from a function namespace to the outside world


### Bring variables from a func namespace to the sideout world

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)    
    
    exec(b, globals().update(locals())) # make sure b can access lst from above

    add1 = locals()['add'] 
    print(add1(5,6))
    add1(5,6)
    pprint(f'locals: {locals()}', width=157)

    # bring variables inside a func to the outside
    globals().update(locals())

test(add)
pprint(add(5,6)) # the original add is override by the add from the function's locals()
pprint(add1(5,6))
print(lst)

"locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
11
("locals: {'func': <function add>, 'a': 1, 'lst': [5, 5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add "
 "at 0x118b54ca0>, 'add1': <function add>}")
11
11
[5, 5, 5, 5]


### globals() in a cell vs globals() in a func

In [ ]:
from fastdebug.utils import tstenv

In [ ]:
len(globals().keys())

87

In [ ]:
globals()['__name__']

'__main__'

In [ ]:
tstenv()

out global env has 20 vars
inner global env has 20 vars
inner local env has 20 vars
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
out env['__name__']: fastdebug.utils
inner env['__name__']: fastdebug.utils


## make a colorful string

In [ ]:
#|export
class dbcolors:
    g = '\033[92m' #GREEN
    y = '\033[93m' #YELLOW
    r = '\033[91m' #RED
    reset = '\033[0m' #RESET COLOR

In [ ]:
#|export
def colorize(cmt, color:str=None):
    if color == "g":
        return dbcolors.g + cmt + dbcolors.reset
    elif color == "y":
        return dbcolors.y + cmt + dbcolors.reset
    elif color == "r":
        return dbcolors.r + cmt + dbcolors.reset
    else: 
        return cmt

In [ ]:
colorize("this is me", "r")

'\x1bthis is me\x1b'

In [ ]:
print(colorize("this is me", "r"))

this is me


## align text to the most right

In [ ]:
#| export
import re

In [ ]:
#| export
def strip_ansi(source):
    return re.sub(r'\033\[(\d|;)+?m', '', source)

In [ ]:
#| export
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [ ]:
alignright("this is me")

                                                                                                                                                   this is me


## print out src code

In [ ]:
#| export
import inspect

### basic version

In [ ]:

def printsrc(src, srclines, cmt):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')

    ccount = 0
    for l in lst:
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else: 
            print('{:<157}'.format(l)) # print out the rest of source code

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
printsrc(foo, "a = 1 + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
                                                                                                                                              this is comment
    b = a + 1                                                                                                                                                
    pass                                                                                                                                                     
                                                

In [ ]:
printsrc(foo, "    a = 1 + 1\n    b = a + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
    b = a + 1================================================================================================================================================
                                                                                                                                              this is comment
    pass                                                                                                                                                     
                                                

### print src with specific number of lines

In [ ]:
len("    this is a code\nthis is another code".split('\n'))
list(map(lambda x: bool(x.strip()), "    this is a code\n    this is another code\n   \n   ".split('\n'))).count(True)

2

In [ ]:
#| export
def printsrc(src, srclines, cmt, expand:int=2):

    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')
    
    # find out the idx of start srclines and end srclines
    startidx = 0
    numsrclines = list(map(lambda x: bool(x.strip()), srclines.split('\n'))).count(True)
    for idx, l in zip(range(len(lst)), lst):
        if bool(l) and l.strip() in srclines:
            startidx = idx
    endidx = startidx + numsrclines - 1
    
    ccount = 0
    for idx, l in zip(range(len(lst)), lst):
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation with # as padding
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                # numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        elif idx < startidx and idx >= startidx - expand:
            print('{:<157}'.format(l)) # print out the rest of source code to the most left

        elif idx <= endidx + expand and idx > endidx:
            print('{:<157}'.format(l)) # print out the rest of source code to the most left
    

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a

In [ ]:
printsrc(foo, "else:", "this is comment", expand=4)

    # this is a pure comment                                                                                                                                 
    if a > 1:                                                                                                                                                
        for i in range(3):                                                                                                                                   
            a = i + 1                                                                                                                                        
    else:====================================================================================================================================================
                                                                                                                                              this is comment
        "this is docs"                              

In [ ]:
printsrc(foo, "return a", "this is comment", expand=1)

    "this is docs"                                                                                                                                           
    return a=================================================================================================================================================
                                                                                                                                              this is comment
                                                                                                                                                             


In [ ]:
printsrc(foo, "    a = 1 + 1\n    b = a + 1", "this is comment")

    else:                                                                                                                                                    
        "this is docs"                                                                                                                                       
        b = a + 1============================================================================================================================================
    return a                                                                                                                                                 
                                                                                                                                                             


more [complex example](./examples/printsrc.ipynb) on printsrc

## dbprint on expression

### basic version

In [ ]:

def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "debug a srcline with one or more expressions with src printed."
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt)
    
    for c in code:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))   

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
def foo(): 
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")
    a = 1 + 1
    pass

In [ ]:
foo()

def foo():                                                                                                                                                   
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")                                                                                       
    a = 1 + 1================================================================================================================================================
                                                                                                                                               this is a test
    pass                                                                                                                                                     
                                                                                                                                                             
                                                    

### insert dbcode and make a new dbfunc

when bringing back splitted lines back, we need add '\n' back to them

In [ ]:
back = ""
for l in inspect.getsource(foo).split('\n'):
    back = back + l
pprint(back, width=157)

'def foo():     dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")    a = 1 + 1    pass'


In [ ]:

def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2 # span 2 lines of srcode up and down from the srcline investigated
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)
    
    # exec the dbsrc to replace the official source code
    exec(dbsrc) # created new foo and saved inside locals()

    
    # check to see whether the new srcode is created
    # print(locals())
    
    # move this new foo into globals, so that outside the cell we can still use it
    globals().update(locals())
    
    return locals()[defaults.orisrc.__name__]
    

### Bring outside namespace variables into exec()

In [ ]:

def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = globals() # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)
    
    # exec the dbsrc to replace the official source code
    # exec(dbsrc) # created new foo and saved inside locals()
    # exec(dbsrc, globals().update(locals())) # make sure b can access lst from above
    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # move this new foo into globals, so that outside the cell we can still use it
    globals().update(locals())
    
    return locals()[defaults.orisrc.__name__]
    

### Bring what inside the func namespace variables to the outside world

In [ ]:

def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = globals() # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]
    

see a [complex example](./examples/dbprint.ipynb) on dbprint

### Adding g = locals() to dbprintinsert to avoid adding env individually

In [ ]:
#| export
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = {} # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n' # adding this line above dbprint line
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
    
    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]
    

## dbprintinsert

### Run and display the inserted dbcodes 
for each srcline under investigation, used inside dbprint

In [ ]:

def dbprintinsert(*codes, **env): 
    for c in codes:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))   
        

In [ ]:
def foo(a):
    a = a*2
    b = a + 1
    c = a * b
    return c

In [ ]:
bool("   ".strip())

False

In [ ]:
dbprint(foo, "b = a + 1", "comment", "a", "a + 1", "a=a")

def foo(a):                                                                                                                                                  
    a = a*2                                                                                                                                                  
    b = a + 1================================================================================================================================================
                                                                                                                                                      comment
    c = a * b                                                                                                                                                
    return c                                                                                                                                                 


<function __main__.foo(a)>

In [ ]:
foo(3) # 

42

In [ ]:
dbprint(foo, "c = a * b", "comment", "b", "b * a", "a=a", "b=b", expand=3)

def foo(a):                                                                                                                                                  
    a = a*2                                                                                                                                                  
    b = a + 1                                                                                                                                                
    c = a * b================================================================================================================================================
                                                                                                                                                      comment
    return c                                                                                                                                                 
                                                    

<function __main__.foo(a)>

In [ ]:
foo(3)

                                                                                                                                                   a => a : 6
                                                                                                                                           a + 1 => a + 1 : 7


42

In [ ]:
dbprint(foo, "b = a + 1\nc = a * b", "comment", "a", "b", "b * a", "a=a", expand=3)

foo(3)

def foo(a):                                                                                                                                                  
    a = a*2                                                                                                                                                  
    b = a + 1================================================================================================================================================
    c = a * b================================================================================================================================================
                                                                                                                                                      comment
                                                                                                                                                             
                                                    

42

In [ ]:
foo1 = dbprint(foo, "b = a + 1", "comment", "a", "a + 1", "a=a")
foo1(3)

def foo(a):                                                                                                                                                  
    a = a*2                                                                                                                                                  
    b = a + 1================================================================================================================================================
                                                                                                                                                      comment
    c = a * b                                                                                                                                                
    return c                                                                                                                                                 
                                                    

42

### use locals() inside the dbsrc code to avoid adding env individually

In [ ]:
#| export
def dbprintinsert(*codes, env={}): 
    for c in codes:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))   
        

In [ ]:
foo1 = dbprint(foo, "b = a + 1", "comment", "a", "a + 1", "env=g")
foo1(3)

def foo(a):                                                                                                                                                  
    a = a*2                                                                                                                                                  
    b = a + 1================================================================================================================================================
                                                                                                                                                      comment
    c = a * b                                                                                                                                                
    return c                                                                                                                                                 
                                                    

42

In [ ]:
from fastcore.meta import delegates
ls = inspect.getsource(delegates).split('\n')
ls = ls[:-1]
ls

['def delegates(to:FunctionType=None, # Delegatee',
 '              keep=False, # Keep `kwargs` in decorated function?',
 '              but:list=None): # Exclude these parameters from signature',
 '    "Decorator: replace `**kwargs` in signature with params from `to`"',
 '    if but is None: but = []',
 '    def _f(f):',
 '        if to is None: to_f,from_f = f.__base__.__init__,f.__init__',
 '        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f',
 "        from_f = getattr(from_f,'__func__',from_f)",
 "        to_f = getattr(to_f,'__func__',to_f)",
 "        if hasattr(from_f,'__delwrap__'): return f",
 '        sig = inspect.signature(from_f)',
 '        sigd = dict(sig.parameters)',
 "        k = sigd.pop('kwargs')",
 '        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()',
 '              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}',
 '        anno = {k:v for

## printrunsrclines() 

It can print out only srclines which actually ran

### Examples

#### simple example

In [ ]:
# def foo(a):
#     if a > 1:
#         a = 1 + 1
#     else:
#         b = a + 1

#### complex example

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
        

In [ ]:
foo(3)

3

### insert a line after each srcline to add idx

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    # if "if" in l or "else" in l or "for" in l:
        
    dbsrc = dbsrc + l + f"\n    srcidx.append({idx})\n" # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
    srcidx.append(3)
        for i in range(3):
    srcidx.append(4)
            a = i + 1
    srcidx.append(5)
    else:
    srcidx.append(6)
        "this is docs"
    srcidx.append(7)
        b = a + 1
    srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

    srcidx.append(11)



### add correct indentation to each inserted line

#### count the indentation for each srcline

In [ ]:
len("    a = 1") - len("    a = 1".strip())

4

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"   # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
    srcidx.append(3)
        for i in range(3):
        srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
    srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

srcidx.append(11)



### indentation special case: if, else, for, def

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

srcidx.append(11)



### remove pure comments or docs from dbsrc
Do not insert line for pure comment or pure "\n"

In [ ]:
from pprint import pprint
for l in srclines:
    pprint(l)

'def foo(a):'
'    "this is docs"'
'    # this is a pure comment'
'    if a > 1:'
'        for i in range(3):'
'            a = i + 1'
'    else:'
'        "this is docs"'
'        b = a + 1'
'    "this is docs"'
'    return a'
''


In [ ]:
"# this is a comment".startswith("#")

True

In [ ]:
"a = 1 # this is comment".startswith("#")

False

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l): # ignore pure '\n'
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)



In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'): # now the dbsrc has no pure comment and pure docs
    print(l)

def foo(a):
    srcidx.append(0)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        b = a + 1
        srcidx.append(8)
    return a
    srcidx.append(10)



In [ ]:
foo??

Signature: foo(a)
Source:   
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
File:      /var/folders/gz/ch3n2mp51m9386sytqf97s6w0000gn/T/ipykernel_56492/3208300118.py
Type:      function


In [ ]:
exec(dbsrc) # give life to dbsrc

In [ ]:
foo??

Signature: foo(a)
Docstring: <no docstring>
File:      Dynamically generated function. No source code available.
Type:      function


In [ ]:
srcidx = [] #used outside the srcode

In [ ]:
foo(3) # run the example using dbsrc
# foo(-1) # run the example using dbsrc
srcidx # Now it should have all the idx whose srclines have run

[0, 3, 4, 5, 4, 5, 4, 5]

### print out the srclines which get run

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx:
        print(l)

def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1


### Make sure all if, else, for get printed

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx or "for" in l or "if" in l or "else" in l:
        print(l)

def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


### Put all together into the function printrunsrclines()

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
        

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))
    fool = locals()['foo']
    pprint(fool(3))
    pprint(locals())

     
    # run = "foo(3)"
    exec("fool(3)")
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(foo)

3
{'addline': 'srcidx.append(11)',
 'dbsrc': 'def foo(a):\n'
          '    srcidx.append(0)\n'
          '    if a > 1:\n'
          '        srcidx.append(3)\n'
          '        for i in range(3):\n'
          '            srcidx.append(4)\n'
          '            a = i + 1\n'
          '            srcidx.append(5)\n'
          '    else:\n'
          '        srcidx.append(6)\n'
          '        b = a + 1\n'
          '        srcidx.append(8)\n'
          '    return a\n'
          '    srcidx.append(10)\n',
 'foo': <function foo>,
 'fool': <function foo>,
 'func': <function foo>,
 'idx': 11,
 'indent': 4,
 'l': '',
 'numindent': 0,
 'srcidx': [0, 3, 4, 5, 4, 5, 4, 5],
 'srclines': ['def foo(a):',
              '    "this is docs"',
              '    # this is a pure comment',
              '    if a > 1:',
              '        for i in range(3):',
              '            a = i + 1',
              '    else:',
              '        "this is docs"',
              '     

#### no more renaming of foo

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec("foo(3)") # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
def foo(a):

    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        b = a + 1
    return a

In [ ]:
printrunsrclines(foo)

[0, 2, 3, 4, 3, 4, 3, 4]
def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


#### add example as a param into the function

In [ ]:
#| export
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(foo, "foo(-1)")

('def foo(a):\n'
 '    srcidx.append(0)\n'
 '    if a > 1:\n'
 '        srcidx.append(2)\n'
 '        for i in range(3):\n'
 '            srcidx.append(3)\n'
 '            a = i + 1\n'
 '            srcidx.append(4)\n'
 '    else:\n'
 '        srcidx.append(5)\n'
 '        b = a + 1\n'
 '        srcidx.append(6)\n'
 '    return a\n'
 '    srcidx.append(7)\n')
[0, 5, 6]
def foo(a):
    if a > 1:
        for i in range(3):
    else:
        b = a + 1


In [ ]:
printrunsrclines(foo, "foo(2)")

('def foo(a):\n'
 '    srcidx.append(0)\n'
 '    if a > 1:\n'
 '        srcidx.append(2)\n'
 '        for i in range(3):\n'
 '            srcidx.append(3)\n'
 '            a = i + 1\n'
 '            srcidx.append(4)\n'
 '    else:\n'
 '        srcidx.append(5)\n'
 '        b = a + 1\n'
 '        srcidx.append(6)\n'
 '    return a\n'
 '    srcidx.append(7)\n')
[0, 2, 3, 4, 3, 4, 3, 4]
def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


#### improve on search for `if`, else, for, def to avoid errors for more examples

In [ ]:

def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

('def alignright(blocks):\n'
 '    srcidx.append(0)\n'
 "    lst = blocks.split('\\n')\n"
 '    srcidx.append(1)\n'
 '    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))\n'
 '    srcidx.append(2)\n'
 '    indent = defaults.margin - maxlen\n'
 '    srcidx.append(3)\n'
 '    for l in lst:\n'
 '        srcidx.append(4)\n'
 "        print(' '*indent + format(l))\n"
 '        srcidx.append(5)\n')
                                                                                                                                                   this is me
[0, 1, 2, 3, 4, 5]
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))


#### remove an empty line with indentation

In [ ]:
lst = """
this is code\n\
     \n\
this is code
""".split('\n')
print(lst)
for l in lst:
    print(bool(l.strip()))

['', 'this is code', '     ', 'this is code', '']
False
True
False
True
False


In [ ]:

def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip()) # how to strip only the left not the right?????
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l.strip()) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): 
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

('def alignright(blocks):\n'
 '    srcidx.append(0)\n'
 "    lst = blocks.split('\\n')\n"
 '    srcidx.append(1)\n'
 '    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))\n'
 '    srcidx.append(2)\n'
 '    indent = defaults.margin - maxlen\n'
 '    srcidx.append(3)\n'
 '    for l in lst:\n'
 '        srcidx.append(4)\n'
 "        print(' '*indent + format(l))\n"
 '        srcidx.append(5)\n')
                                                                                                                                                   this is me
[0, 1, 2, 3, 4, 5]
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))


In [ ]:
pprint(inspect.getsource(printsrc))

('def printsrc(src, srclines, cmt, expand:int=2):\n'
 '\n'
 '    # convert the source code of the function into a list of strings '
 "splitted by '\\n'\n"
 "    lst = inspect.getsource(src).split('\\n')\n"
 '    \n'
 '    # find out the idx of start srclines and end srclines\n'
 '    startidx = 0\n'
 '    numsrclines = list(map(lambda x: bool(x.strip()), '
 "srclines.split('\\n'))).count(True)\n"
 '    for idx, l in zip(range(len(lst)), lst):\n'
 '        if bool(l) and l.strip() in srclines:\n'
 '            startidx = idx\n'
 '    endidx = startidx + numsrclines - 1\n'
 '    \n'
 '    ccount = 0\n'
 '    for idx, l in zip(range(len(lst)), lst):\n'
 '        if bool(l) and l.strip() in srclines:# print out the srcline under '
 'investigation with # as padding\n'
 "            print('{:=<157}'.format(l))\n"
 '            ccount = ccount + 1\n'
 '\n'
 '            if bool(cmt): # print out comment at the end of the srclines '
 'under investigation\n'
 '                # numsrclines = le

In [ ]:
#| export
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip()) # how to strip only the left not the right?????
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l.strip()) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): 
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    # pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    # print(srcidx)

    # pprint(srclines)
    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(printsrc, 'printsrc(foo, "    else:\\n    b = a + 1", "this is comment")') # make sure to use \\n not \n

            a = i + 1                                                                                                                                        
    else:====================================================================================================================================================
        b = a + 1============================================================================================================================================
                                                                                                                                              this is comment
                                                                                                                                                             
def printsrc(src, srclines, cmt, expand:int=2):
    lst = inspect.getsource(src).split('\n')
    startidx = 0
    numsrclines = list(map(lambda x: bool(x.strip()), srclines.split('\n'))).count(True)
    for idx

In [ ]:
printrunsrclines(printsrc, 'printsrc(foo, "    else:\\n    b = a + 1", "")') # make sure to use \\n not \n

            a = i + 1                                                                                                                                        
    else:====================================================================================================================================================
        b = a + 1============================================================================================================================================
                                                                                                                                                             
def printsrc(src, srclines, cmt, expand:int=2):
    lst = inspect.getsource(src).split('\n')
    startidx = 0
    numsrclines = list(map(lambda x: bool(x.strip()), srclines.split('\n'))).count(True)
    for idx, l in zip(range(len(lst)), lst):
        if bool(l) and l.strip() in srclines:
            startidx = idx
    endidx = startidx + numsrclines - 1
    ccount 

### more difficult examples to test printrunsrc()

#|hide
## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

#|hide
## Send to Obsidian

In [ ]:
#| hide
!jupytext --to md /Users/Natsume/Documents/fastdebug/00_core.ipynb
!mv /Users/Natsume/Documents/fastdebug/00_core.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/

[jupytext] Reading /Users/Natsume/Documents/fastdebug/00_core.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/fastdebug/00_core.md


In [ ]:
#| hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/*.ipynb' matched no files
[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/meta/*.ipynb' matched no files
[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/findings/*.ipynb' matched no files
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/index.ipynb to markdown
[NbConvertApp] Writing 306 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/FooGetSigInit.ipynb to markdown
[NbConvertApp] Writing 39294 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/FooGetSigInit.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/00_core.ipynb to markdown
[NbConvertApp] Writing 76407 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/00_core.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fa